In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd
import os
import fiona

In [2]:
cnty = {'Alameda': 1.0,
'Contra Costa': 13.0,
'Marin': 41.0,
'Napa': 55.0,
'San Francisco': 75.0,
'San Mateo': 81.0,
'Santa Clara': 85.0,
'Solano': 95.0,
'Sonoma': 97.0}

ctyMap = pd.DataFrame(cnty.items(), columns=['ctyName', 'ctyCode'])

## 0 Data sources

In [3]:
print(fiona.listlayers(r'C:\Users\ywang\Documents\ArcGIS\Projects\PLU_analysis\PLU_analysis.gdb'))

# Input files
## Pacel 10
p10_raw = gpd.read_file(r'C:\Users\ywang\Documents\ArcGIS\Projects\PLU_analysis\PLU_analysis.gdb', layer='p10_table')

## parcel10 to pba40 basezoning code
pz10 = pd.read_csv(r'C:\Users\ywang\Box\Mine\1_UrbanSim\2020_03_06_zoning_parcels.csv')

## pba40 basezoning plu
plu10 = pd.read_csv('C:/Users/ywang/Documents/GitHub/bayarea_urbansim/data/zoning_lookup.csv')

## pba50 basezoning PLU
p10_plu50_raw = gpd.read_file(r'C:\Users\ywang\Documents\ArcGIS\Projects\PLU_analysis\PLU_analysis.gdb', layer='p10_boc_v3_geo_tbl_20200311')

## planned zoning scenarios
zmods = pd.read_csv('C:/Users/ywang/Box/Mine/1_UrbanSim/03_06_2020_parcels_geography.csv')

## Building data to decide parcel status
blg10 = pd.read_csv('blg10.csv')

['p10_table', 'p10_boc_v3_geo_tbl_20200311']


## 1 Merge data sets

### 1.1 P10 parcel zoining designations

In [4]:
# parcel geometry
p10 = p10_raw[['PARCEL_ID','APN','geom_id_s','COUNTY_ID','jurisdiction',
                           'ACRES','LAND_VALUE','pda_id','zoningmodcat']]
print(p10.shape)
display(p10.head())

# pacel to zoning code mapping
print(pz10.shape)
display(pz10.head())

p10_z10 = p10.merge(pz10, on = 'PARCEL_ID', how = 'left')
print(p10_z10.shape)
display(p10_z10.head())

# Check Number of parcels missing zoning designation
z10_missing = p10_z10.loc[p10_z10['nodev_pba40'].isnull()]
print(z10_missing.shape[0])
print(z10_missing.shape[0]/pz10.shape[0])

(1956208, 9)


,PARCEL_ID,APN,geom_id_s,COUNTY_ID,jurisdiction,ACRES,LAND_VALUE,pda_id,zoningmodcat
0,229116.0,099 029001700,10305106092872,1.0,41992,3.360520,0.0,NA,41992NANANANA
1,244166.0,099B540210200,11107351665227,1.0,41992,1.294423,0.0,NA,41992NANANANA
2,202378.0,085A643106000,11030175960628,1.0,33000,14.993605,6036500.0,NA,33000NANANANA
3,2004420.0,141-100-012,6381677629073,97.0,00097,316.247146,179954.0,NA,00097NANANANA
4,340332.0,525 166004800,314875459798,1.0,26000,0.621275,0.0,NA,26000NAb1NANA


(1950733, 11)


,Unnamed: 0,geom_id,zoning_id,zoning,juris,prop,tablename,nodev_pba40,PARCEL_ID,nodev,juris_id
0,0,1846247885201,12202.0,115 - Residential 0-4 du/ac,102,100,alamedacountygp2006db,0,191124.0,0.0,uala
1,1,11768793521677,12204.0,115 - Residential 9-17 du/ac,102,100,alamedacountygp2006db,0,197219.0,0.0,uala
2,2,807545210880,12204.0,115 - Residential 9-17 du/ac,102,100,alamedacountygp2006db,0,197218.0,0.0,uala
3,3,8785012057974,12204.0,115 - Residential 9-17 du/ac,102,100,alamedacountygp2006db,0,188301.0,0.0,uala
4,4,14057552282712,12204.0,115 - Residential 9-17 du/ac,102,100,alamedacountygp2006db,0,188939.0,0.0,uala


(1956208, 19)


,PARCEL_ID,APN,geom_id_s,COUNTY_ID,jurisdiction,ACRES,LAND_VALUE,pda_id,zoningmodcat,Unnamed: 0,geom_id,zoning_id,zoning,juris,prop,tablename,nodev_pba40,nodev,juris_id
0,229116.0,099 029001700,10305106092872,1.0,41992,3.360520,0.0,NA,41992NANANANA,1771381.0,1.030511e+13,60126.0,107 - Urban Low Residential UL2,-9999.0,100.0,plu06,0.0,0.0,livr
1,244166.0,099B540210200,11107351665227,1.0,41992,1.294423,0.0,NA,41992NANANANA,1362648.0,1.110735e+13,11903.0,GP-ULM,99.0,100.0,livermoregeneralplan,0.0,0.0,livr
2,202378.0,085A643106000,11030175960628,1.0,33000,14.993605,6036500.0,NA,33000NANANANA,307258.0,1.103018e+13,11803.0,LDR,98.0,100.0,hayward_gp_landuse,0.0,0.0,hayw
3,2004420.0,141-100-012,6381677629073,97.0,00097,316.247146,179954.0,NA,00097NANANANA,1737625.0,6.381678e+12,12975.0,LEA240,109.0,100.0,sonomacountygeneralplan,0.0,0.0,uson
4,340332.0,525 166004800,314875459798,1.0,26000,0.621275,0.0,NA,26000NAb1NANA,273989.0,3.148755e+11,2511.0,RESM4,5.0,100.0,fremontgeneralplan,0.0,1.0,frem


5476
0.002807149927745109


### 1.2 parcel 10 with PBA40 zoning code PLU

In [5]:
# check duplicates in zoning id
plu10['id'] = plu10['id'].apply(lambda x: float(x))
plu10['jz_o'] = plu10['city'].str.cat(plu10['name'],sep=" ")
print(plu10.shape[0], len(plu10.id.unique()), len(plu10.jz_o.unique()))

# relabel p10 land plu info (used in PBA40)
cols = [i+'_10' for i in list(plu10)]
plu10.columns = cols
display(plu10.head())

# merge PBA40 plu to p10
p10_plu10 = p10_z10.merge(plu10, left_on = 'zoning_id', right_on = 'id_10', how = 'left')
display(p10_plu10.head())

# Check number of p10 records failed to find a matching PLU
display(p10_plu10.loc[p10_plu10['jz_o_10'].isnull()])
print(p10_plu10.loc[p10_plu10['jz_o_10'].isnull()].shape[0] / p10_z10.shape[0])

5156 5156 4536


,id_10,juris_10,city_10,name_10,max_far_10,max_height_10,max_dua_10,max_du_per_parcel_10,HS_10,HT_10,...,IL_10,IW_10,IH_10,RS_10,RB_10,MR_10,MT_10,ME_10,plandate_10,jz_o_10
0,2101.0,1.0,Albany,RHD,0.50,35.0,9.0,NaN,1,0,...,0,0,0,0,0,0,0,0,NaN,Albany RHD
1,2102.0,1.0,Albany,R-1,0.55,28.0,12.0,NaN,1,0,...,0,0,0,0,0,0,0,0,NaN,Albany R-1
2,2103.0,1.0,Albany,R-2,0.55,35.0,35.0,NaN,1,1,...,0,0,0,0,0,0,0,0,NaN,Albany R-2
3,2104.0,1.0,Albany,R-3,1.50,35.0,63.0,NaN,1,1,...,0,0,0,0,0,0,0,0,NaN,Albany R-3
4,2105.0,1.0,Albany,R.4,NaN,NaN,87.0,NaN,0,0,...,0,0,0,0,0,0,0,0,NaN,Albany R.4


,PARCEL_ID,APN,geom_id_s,COUNTY_ID,jurisdiction,ACRES,LAND_VALUE,pda_id,zoningmodcat,Unnamed: 0,...,IL_10,IW_10,IH_10,RS_10,RB_10,MR_10,MT_10,ME_10,plandate_10,jz_o_10
0,229116.0,099 029001700,10305106092872,1.0,41992,3.360520,0.0,NA,41992NANANANA,1771381.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,Livermore 107 - Urban Low Residential UL2
1,244166.0,099B540210200,11107351665227,1.0,41992,1.294423,0.0,NA,41992NANANANA,1362648.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,Livermore GP-ULM
2,202378.0,085A643106000,11030175960628,1.0,33000,14.993605,6036500.0,NA,33000NANANANA,307258.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,Hayward LDR
3,2004420.0,141-100-012,6381677629073,97.0,00097,316.247146,179954.0,NA,00097NANANANA,1737625.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,Unincorporated Sonoma LEA240
4,340332.0,525 166004800,314875459798,1.0,26000,0.621275,0.0,NA,26000NAb1NANA,273989.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,Fremont RESM4


,PARCEL_ID,APN,geom_id_s,COUNTY_ID,jurisdiction,ACRES,LAND_VALUE,pda_id,zoningmodcat,Unnamed: 0,...,IL_10,IW_10,IH_10,RS_10,RB_10,MR_10,MT_10,ME_10,plandate_10,jz_o_10
1185,580792.0,271333,14613597454578,13.0,16000,0.013816,0.000000e+00,NA,16000NANANANA,1322496.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1637,255073.0,415 000100600,15298997157075,1.0,00001,0.007685,0.000000e+00,NA,00001NANANANA,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1750,1323823.0,15846302,261407886174,85.0,49670,2.773525,3.467860e+07,MVW2,49670MVW2cr2NANA,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1833,2054503.0,,2054503,0.0,00085,0.000000,0.000000e+00,NA,00085NANANANA,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1956,1220739.0,090160500,12373840766913,81.0,17918,0.059405,0.000000e+00,NA,17918NANANANA,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1955104,651123.0,334654,13367406559562,13.0,57456,0.115417,2.306800e+04,PIT1,57456PIT1bart3NANA,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1955850,680555.0,44228,9247454957509,13.0,49187,0.609596,0.000000e+00,NA,49187NANANANA,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1955895,1411792.0,28213001,3484331839412,85.0,68000,7.325086,3.932012e+06,SJO14,68000SJO14lrt3NANA,582835.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1956145,1414620.0,28422021,8329617969303,85.0,68000,2.655408,0.000000e+00,NA,68000NAlrt3NANA,1305453.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


0.0034745793903306807


### 1.3 P10 with BASIS BOC

In [6]:
plu50 = p10_plu50_raw.loc[:,['PARCEL_ID','me','mt', 'mr', 'rb', 'rs', 'ih', 'iw', 'il', 'sc', 'ho', 'of', 'hm', 'ht', 'hs',
                       'max_height','max_dua','max_far','zn_jurisdiction','zn_county']]

print(plu50.shape)

# relabel BASIS land plu info (to use in PBA50)
cols2 = [i+'_18' for i in list(plu50)]
plu50.columns = cols2
display(plu50.head())

# merge PBA50 plu to p10
p10_plus = p10_plu10.merge(plu50, left_on = 'PARCEL_ID', right_on = 'PARCEL_ID_18', how = 'left')
print(p10_plus.shape)

p10_plus.drop(columns = ['zoning','tablename','Unnamed: 0','id_10','name_10','plandate_10','jz_o_10','PARCEL_ID_18'],inplace = True)
display(p10_plus.head())

C:\Users\ywang\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\indexing.py:1418: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#deprecate-loc-reindex-listlike
  return self._getitem_tuple(key)


(1956208, 20)


,PARCEL_ID_18,me_18,mt_18,mr_18,rb_18,rs_18,ih_18,iw_18,il_18,sc_18,ho_18,of_18,hm_18,ht_18,hs_18,max_height_18,max_dua_18,max_far_18,zn_jurisdiction_18,zn_county_18
0,229116.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,None,None,None,NaN,NaN
1,244166.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,35.000000000000000,14.000000000000000,0.350000000000000,NaN,NaN
2,202378.0,1,1,0,0,1,0,1,1,1,0,1,0,0,0,None,None,None,NaN,NaN
3,2004420.0,1,1,1,0,1,0,1,1,0,1,1,1,1,1,35.000000000000000,0.016660000000000,None,NaN,NaN
4,340332.0,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,NaN,NaN


(1956208, 63)


,PARCEL_ID,APN,geom_id_s,COUNTY_ID,jurisdiction,ACRES,LAND_VALUE,pda_id,zoningmodcat,geom_id,...,ho_18,of_18,hm_18,ht_18,hs_18,max_height_18,max_dua_18,max_far_18,zn_jurisdiction_18,zn_county_18
0,229116.0,099 029001700,10305106092872,1.0,41992,3.360520,0.0,NA,41992NANANANA,1.030511e+13,...,0,0,0,0,0,None,None,None,NaN,NaN
1,244166.0,099B540210200,11107351665227,1.0,41992,1.294423,0.0,NA,41992NANANANA,1.110735e+13,...,0,0,0,0,0,35.000000000000000,14.000000000000000,0.350000000000000,NaN,NaN
2,202378.0,085A643106000,11030175960628,1.0,33000,14.993605,6036500.0,NA,33000NANANANA,1.103018e+13,...,0,1,0,0,0,None,None,None,NaN,NaN
3,2004420.0,141-100-012,6381677629073,97.0,00097,316.247146,179954.0,NA,00097NANANANA,6.381678e+12,...,1,1,1,1,1,35.000000000000000,0.016660000000000,None,NaN,NaN
4,340332.0,525 166004800,314875459798,1.0,26000,0.621275,0.0,NA,26000NAb1NANA,3.148755e+11,...,None,None,None,None,None,None,None,None,NaN,NaN


### 1.4 Bring in Building data (b10) to determine parcel characteristics

In [7]:
print(blg10.shape[0], len(blg10.building_id.unique()), len(blg10.parcel_id.unique()))
display(blg10.head())

# Assign parcel characteristics

# merge builing and parcel data w/ Outer-join
b10_p10 = blg10.merge(p10[['PARCEL_ID']],left_on = 'parcel_id',right_on = 'PARCEL_ID', how = 'outer')
print(b10_p10.shape)

# sum all values for multiple buildings within one parcel
pb10_v = b10_p10.groupby(['PARCEL_ID'])['improvement_value','residential_units','residential_sqft','non_residential_sqft',
                                      'building_sqft','redfin_sale_price','costar_rent'].sum().reset_index()

# chose the earliest built year for multiple buildings within one parcel
pb10_yr = b10_p10.groupby(['PARCEL_ID'])['year_built','building_id'].min().reset_index()

# parcel vacancy based on building type
b10_p10['dType'] = b10_p10['development_type_id']
blg10.loc[blg10['development_type_id'] == 0, 'dType'] = 'Vacant'
blg10.loc[blg10['development_type_id'] == 15, 'dType'] = 'Vacant'
pb10_vacent = b10_p10.loc[b10_p10['dType'] == 'Vacant'][['PARCEL_ID','dType']]

# merge
pb10_temp = pb10_v.merge(pb10_yr, on = 'PARCEL_ID', how = 'left').merge(pb10_vacent, on = 'PARCEL_ID', how = 'left')
print(pb10_temp.shape)
pb10_plus = p10_plus.merge(pb10_temp, on = 'PARCEL_ID', how = 'left')

# Investment-land ratio
pb10_plus['ILR'] = pb10_plus['improvement_value'] / pb10_plus['LAND_VALUE']
pb10_plus.loc[pb10_plus['LAND_VALUE'] == 0, 'ILR'] = 'n/a'

# Vacant parcels
pb10_plus['vacant'] = np.where((pb10_plus['building_id'].isnull()) | (pb10_plus['dType'] == 'Vacant') | 
                          ((pb10_plus['improvement_value'] == 0) & (pb10_plus['residential_units'] == 0) & 
                             (pb10_plus['residential_sqft'] == 0) & (pb10_plus['non_residential_sqft'] == 0) &
                             (pb10_plus['building_sqft'] == 0)), 'vacant', 'nonVacant')

pb10_plus.head()

1843351 1843351 1843292


,Unnamed: 0,building_id,parcel_id,development_type_id,improvement_value,residential_units,residential_sqft,sqft_per_unit,non_residential_sqft,building_sqft,...,res_price_per_sqft,stories,year_built,redfin_sale_price,redfin_sale_year,redfin_home_type,costar_property_type,costar_rent,id,geometry
0,0,1,742974,1,0.00,1,2029,2029.42425,0,2029.42425,...,302.769751,1,1945,NaN,NaN,NaN,NaN,NaN,1,NaN
1,1,2,744961,1,0.00,1,2029,2029.42425,0,2029.42425,...,254.429279,1,1965,NaN,NaN,NaN,NaN,NaN,2,NaN
2,2,3,1442641,1,53262.87,1,1568,1568.00000,0,1568.00000,...,183.474166,1,1964,NaN,NaN,NaN,NaN,NaN,3,NaN
3,3,4,190969,2,245000.00,0,0,1266.00000,1595,1266.00000,...,0.000000,2,1992,340000.0,2003.0,Condo/Coop,NaN,NaN,4,NaN
4,4,5,308709,2,283500.00,0,0,1513.00000,1513,1513.00000,...,0.000000,1,1978,442000.0,2004.0,Condo/Coop,NaN,NaN,5,NaN


(1956269, 22)


C:\Users\ywang\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\ops\__init__.py:1115: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)


(1956208, 10)


,PARCEL_ID,APN,geom_id_s,COUNTY_ID,jurisdiction,ACRES,LAND_VALUE,pda_id,zoningmodcat,geom_id,...,residential_units,residential_sqft,non_residential_sqft,building_sqft,redfin_sale_price,year_built,building_id,dType,ILR,vacant
0,229116.0,099 029001700,10305106092872,1.0,41992,3.360520,0.0,NA,41992NANANANA,1.030511e+13,...,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,n/a,vacant
1,244166.0,099B540210200,11107351665227,1.0,41992,1.294423,0.0,NA,41992NANANANA,1.110735e+13,...,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,n/a,vacant
2,202378.0,085A643106000,11030175960628,1.0,33000,14.993605,6036500.0,NA,33000NANANANA,1.103018e+13,...,20.0,101000.0,0.0,101000.0,1007250.0,2009.0,15681.0,NaN,0,nonVacant
3,2004420.0,141-100-012,6381677629073,97.0,00097,316.247146,179954.0,NA,00097NANANANA,6.381678e+12,...,0.0,0.0,0.0,0.0,0.0,1965.0,17798.0,NaN,0.812491,nonVacant
4,340332.0,525 166004800,314875459798,1.0,26000,0.621275,0.0,NA,26000NAb1NANA,3.148755e+11,...,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,n/a,vacant


### 1.5 Bring in zoning scenarios data

In [8]:
print(zmods.shape)
display(zmods.head())

# merge parcel data with zoning mods
pb10_plus_zmods = pb10_plus.merge(zmods, on = 'geom_id', how = 'left')
print(pb10_plus_zmods.shape)
display(pb10_plus_zmods.head())

(1956208, 26)


,geom_id,jurisdiction_id_old,pda_id,tpp_id,exp_id,opp_id,zoningmodcat,perffoot,perfarea,urbanized,...,juris_id,gg_id,tra_id,sesit_id,ppa_id,exp2020_id,exsfd_id,pba50zoningmodcat,nodev,jurisdiction_id
0,10305106092872,41992,NaN,NaN,NaN,NaN,41992NANANANA,1,0,1,...,livr,NaN,NaN,HRADR,NaN,in,NaN,livrNANAHRADRNAinNA,0,41992
1,11107351665227,41992,NaN,NaN,NaN,NaN,41992NANANANA,1,0,1,...,livr,NaN,NaN,DR,NaN,in,NaN,livrNANADRNAinNA,0,41992
2,11030175960628,33000,NaN,NaN,NaN,NaN,33000NANANANA,1,0,0,...,hayw,NaN,NaN,NaN,NaN,in,NaN,haywNANANANAinNA,0,33000
3,6381677629073,97,NaN,NaN,NaN,NaN,00097NANANANA,0,0,0,...,uson,NaN,NaN,DR,NaN,out,NaN,usonNANADRNAoutNA,0,97
4,314875459798,26000,NaN,b1,NaN,NaN,26000NAb1NANA,1,1,1,...,frem,NaN,NaN,HRADR,NaN,in,NaN,fremNANAHRADRNAinNA,1,26000


(1956208, 91)


,PARCEL_ID_x,APN,geom_id_s,COUNTY_ID,jurisdiction,ACRES,LAND_VALUE,pda_id_x,zoningmodcat_x,geom_id,...,juris_id_y,gg_id,tra_id,sesit_id,ppa_id,exp2020_id,exsfd_id,pba50zoningmodcat,nodev_y,jurisdiction_id
0,229116.0,099 029001700,10305106092872,1.0,41992,3.360520,0.0,NA,41992NANANANA,1.030511e+13,...,livr,NaN,NaN,HRADR,NaN,in,NaN,livrNANAHRADRNAinNA,0.0,41992.0
1,244166.0,099B540210200,11107351665227,1.0,41992,1.294423,0.0,NA,41992NANANANA,1.110735e+13,...,livr,NaN,NaN,DR,NaN,in,NaN,livrNANADRNAinNA,0.0,41992.0
2,202378.0,085A643106000,11030175960628,1.0,33000,14.993605,6036500.0,NA,33000NANANANA,1.103018e+13,...,hayw,NaN,NaN,NaN,NaN,in,NaN,haywNANANANAinNA,0.0,33000.0
3,2004420.0,141-100-012,6381677629073,97.0,00097,316.247146,179954.0,NA,00097NANANANA,6.381678e+12,...,uson,NaN,NaN,DR,NaN,out,NaN,usonNANADRNAoutNA,0.0,97.0
4,340332.0,525 166004800,314875459798,1.0,26000,0.621275,0.0,NA,26000NAb1NANA,3.148755e+11,...,frem,NaN,NaN,HRADR,NaN,in,NaN,fremNANAHRADRNAinNA,1.0,26000.0


## 2 Capacity statistics

### 2.1 Caculate Build out capacity for each parcel

In [9]:
# select needed fields
plu_main = pb10_plus_zmods.loc[:,['COUNTY_ID','juris_id_y','zoning_id','geom_id_s','ACRES',
                    'max_far_10','max_dua_10','max_dua_18','max_far_18',
                    'HS_10','HT_10','HM_10','OF_10','HO_10','SC_10','IL_10','IW_10','IH_10','RS_10','RB_10','MR_10','MT_10','ME_10',
                    'me_18','mt_18','mr_18','rb_18','rs_18','ih_18','iw_18','il_18','sc_18','ho_18','of_18','hm_18','ht_18','hs_18',
                    'year_built','ILR','vacant','pba50zoningmodcat','nodev_pba40','nodev_pba50']]

# Convert all types to numeric to enable calculation
l = ['HS_10','HT_10','HM_10','OF_10','HO_10','SC_10','IL_10','IW_10','IH_10','RS_10','RB_10','MR_10','MT_10','ME_10',
     'me_18','mt_18','mr_18','rb_18','rs_18','ih_18','iw_18','il_18','sc_18','ho_18','of_18','hm_18','ht_18','hs_18',
     'max_far_10','max_dua_10','max_dua_18','max_far_18']

for i in l:
    plu_main[i] = pd.to_numeric(plu_main[i], errors='coerce')

# fill 'NaN' with 0
plu_main.update(plu_main[l].fillna(0))

In [10]:
# Calculate BOC Based on PBA40 PLU

## A parcel is 'allowNonRes' is at least one of the non-residential development types is allowed; then FAR calculations apply
plu_main['allowNonRes_10'] = plu_main[['OF_10','HO_10','SC_10','IL_10','IW_10','IH_10','RS_10','RB_10','MR_10','MT_10','ME_10']].sum(axis=1) > 0

## A parcel is 'allowRes' is at least one of the residential development types is allowed; then DUA calculations apply
plu_main['allowRes_10'] = plu_main[['HS_10','HT_10','HM_10']].sum(axis=1) > 0

plu_main['units_10'] = plu_main['ACRES'] * plu_main['max_dua_10']
plu_main.loc[(plu_main['allowRes_10'] is False) | (plu_main['nodev_pba40'] == 1) ,'units_10'] = 0
plu_main['sf_10'] = plu_main['ACRES'] * plu_main['max_far_10'] * 43560
plu_main.loc[(plu_main['allowNonRes_10'] is False) | (plu_main['nodev_pba50'] == 1),'sf_10'] = 0


# Calculate BOC Based on BASIS PLU

plu_main['allowNonRes_18'] = plu_main[['of_18','ho_18','sc_18','il_18','iw_18','ih_18','rs_18','rb_18','mr_18','mt_18','me_18']].sum(axis=1) > 0
plu_main['allowRes_18'] = plu_main[['hs_18','ht_18','hm_18']].sum(axis=1) > 0

plu_main['units_18'] = plu_main['ACRES'] * plu_main['max_dua_18']
plu_main.loc[(plu_main['allowRes_18'] is False) | (plu_main['hs_18'] == 1) ,'units_18'] = 0
plu_main['sf_18'] = plu_main['ACRES'] * plu_main['max_far_18'] * 43560
plu_main.loc[(plu_main['allowNonRes_18'] is False) | (plu_main['hs_18'] == 1),'sf_18'] = 0

display(plu_main)

,COUNTY_ID,juris_id_y,zoning_id,geom_id_s,ACRES,max_far_10,max_dua_10,max_dua_18,max_far_18,HS_10,...,nodev_pba40,nodev_pba50,allowNonRes_10,allowRes_10,units_10,sf_10,allowNonRes_18,allowRes_18,units_18,sf_18
0,1.0,livr,60126.0,10305106092872,3.360520,0.0,2.00000,0.00000,0.00,1.0,...,0.0,NaN,False,True,6.721041,0.000000,False,False,0.000000,0.000000
1,1.0,livr,11903.0,11107351665227,1.294423,0.0,3.00000,14.00000,0.35,0.0,...,0.0,NaN,False,True,3.883268,0.000000,False,False,18.121919,19734.769419
2,1.0,hayw,11803.0,11030175960628,14.993605,0.0,8.70000,0.00000,0.00,1.0,...,0.0,NaN,False,True,130.444362,0.000000,True,False,0.000000,0.000000
3,97.0,uson,12975.0,6381677629073,316.247146,0.0,0.00417,0.01666,0.00,1.0,...,0.0,NaN,False,True,1.318751,0.000000,True,True,0.000000,0.000000
4,1.0,frem,2511.0,314875459798,0.621275,0.0,23.00000,0.00000,0.00,1.0,...,0.0,NaN,True,True,14.289334,0.000000,False,False,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1956203,13.0,anti,10204.0,17158666132196,0.071424,0.0,10.00000,0.00000,0.00,1.0,...,0.0,NaN,True,True,0.714239,0.000000,True,True,0.000000,0.000000
1956204,13.0,conc,10702.0,16389503450045,0.137534,0.0,10.00000,6.00000,0.00,1.0,...,0.0,NaN,False,True,1.375336,0.000000,False,True,0.000000,0.000000
1956205,41.0,nova,8213.0,1496694834659,0.019658,0.4,0.00000,4.35600,0.40,0.0,...,0.0,NaN,True,False,0.000000,342.523479,True,False,0.085631,342.523479
1956206,13.0,ucnc,12302.0,10694584892329,0.254764,0.0,2.90000,0.00000,0.00,1.0,...,0.0,NaN,False,True,0.738815,0.000000,False,False,0.000000,0.000000


### 2.2 Build out capacity at jurisdiction and county levels

In [11]:
# BOC by jurisdiction function
def boc_j(df):
    boc_j = df.groupby(['juris_id_y'])['ACRES','units_10','units_18','sf_10','sf_18'].sum()

    boc_j['unit_diff'] = boc_j['units_18'] - boc_j['units_10']
    boc_j['sqft_diff'] = boc_j['sf_18'] - boc_j['sf_10']
    boc_j['unit_diff_pct'] = boc_j['unit_diff'] / boc_j['units_10']
    boc_j['sqft_diff_pct'] = boc_j['sqft_diff'] / boc_j['sf_10']

    for i in ['units_10','units_18','unit_diff','sf_10','sf_18','sqft_diff']:
        boc_j[i] = boc_j[i].apply(lambda x: f'{int(x):,}')
    display(boc_j)
    return boc_j

# BOC by county function
def boc_c(df):
    boc_cty = df.groupby(['COUNTY_ID'])['ACRES','units_10','units_18','sf_10','sf_18'].sum()
    boc_cty['unit_diff'] = boc_cty['units_18'] - boc_cty['units_10']
    boc_cty['sqft_diff'] = boc_cty['sf_18'] - boc_cty['sf_10']
    boc_cty['unit_diff_pct'] = boc_cty['unit_diff'] / boc_cty['units_10']
    boc_cty['sqft_diff_pct'] = boc_cty['sqft_diff'] / boc_cty['sf_10']

    for i in ['units_10','units_18','unit_diff','sf_10','sf_18','sqft_diff']:
        boc_cty[i] = boc_cty[i].apply(lambda x: f'{int(x):,}')

    boc_cty = boc_cty.reset_index()
    boc_cty = boc_cty.loc[boc_cty['COUNTY_ID'] > 0]

    boc_c = boc_cty.merge(ctyMap, left_on = 'COUNTY_ID', right_on = 'ctyCode', how = 'left')
    boc_c.drop(columns = ['ctyCode'],inplace=True)
    display(boc_c)
    return boc_c

In [12]:
# all parcels statistics
all_boc_j = boc_j(plu_main)
all_boc_j.to_csv('all_boc_jurisdiction.csv')

all_boc_c = boc_c(plu_main)
all_boc_c.to_csv('all_boc_county.csv')

,ACRES,units_10,units_18,sf_10,sf_18,unit_diff,sqft_diff,unit_diff_pct,sqft_diff_pct
juris_id_y,,,,,,,,,
alam,6576.815744,"34,053","10,661","58,997,768",0,"-23,392","-58,997,768",-0.686921,-1.000000
alba,2685.790749,"9,803",636,"17,688,038","7,601,644","-9,167","-10,086,394",-0.935080,-0.570238
amer,3459.100475,"6,842",320,"40,680,354","19,360,101","-6,521","-21,320,253",-0.953099,-0.524092
anti,15626.623830,"79,272","5,118","160,767,171","3,739,633","-74,154","-157,027,538",-0.935434,-0.976739
athe,2884.446231,"2,518",176,"20,591,884","160,858","-2,341","-20,431,025",-0.930025,-0.992188
...,...,...,...,...,...,...,...,...,...
vall,18923.391859,"59,916",0,"113,745,786",0,"-59,916","-113,745,786",-0.999999,-1.000000
walc,11199.444875,"50,396","16,559","53,214,620","7,416,275","-33,836","-45,798,345",-0.671413,-0.860635
wind,4000.762589,"14,103","19,399","10,497,930","29,987,905","5,295","19,489,974",0.375476,1.856554


,COUNTY_ID,ACRES,units_10,units_18,sf_10,sf_18,unit_diff,sqft_diff,unit_diff_pct,sqft_diff_pct,ctyName
0,1.0,494335.528614,"1,132,680","619,373","2,360,201,919","1,362,301,521","-513,306","-997,900,397",-0.453179,-0.422803,Alameda
1,13.0,446722.616106,"768,422","252,515","1,059,290,704","676,371,796","-515,907","-382,918,908",-0.671385,-0.361486,Contra Costa
2,41.0,371976.214899,"223,537","483,630","191,450,982","1,475,648,214","260,093","1,284,197,231",1.163535,6.707708,Marin
3,55.0,495755.032111,"464,635","38,652","778,118,312","768,254,497","-425,983","-9,863,815",-0.916812,-0.012676,Napa
4,75.0,22683.294626,"411,420","112,517","1,405,478,871","46,548,778","-298,902","-1,358,930,092",-0.726515,-0.966880,San Francisco
5,81.0,336923.887211,"394,233","187,183","979,620,948","498,798,150","-207,050","-480,822,798",-0.525197,-0.490825,San Mateo
6,85.0,797516.101744,"1,028,659","395,093","7,985,213,594","721,477,602,297","-633,565","713,492,388,702",-0.615914,89.351697,Santa Clara
7,95.0,529122.858613,"239,209","92,854","690,367,399","382,076,997","-146,355","-308,290,401",-0.611827,-0.446560,Solano
8,97.0,993067.346741,"299,285","156,142","1,679,114,336","229,999,174","-143,142","-1,449,115,162",-0.478282,-0.863024,Sonoma


In [13]:
# vacant parcel statistics

p_vac = plu_main.loc[plu_main.vacant == 'vacant']

vac_boc_j = boc_j(p_vac)
vac_boc_j.to_csv('vac_boc_jurisdiction.csv')

vac_boc_c = boc_c(p_vac)
vac_boc_c.to_csv('vac_boc_county.csv')

,ACRES,units_10,units_18,sf_10,sf_18,unit_diff,sqft_diff,unit_diff_pct,sqft_diff_pct
juris_id_y,,,,,,,,,
alam,1248.085181,"1,700",301,"10,133,514",0,"-1,398","-10,133,514",-0.822771,-1.000000
alba,432.557933,80,0,"452,734","7,038,877",-80,"6,586,143",-1.000000,14.547490
amer,532.289077,446,72,"5,589,327","4,791,424",-374,"-797,902",-0.838452,-0.142755
anti,7229.729103,"31,192","1,720","104,765,225","2,372,923","-29,472","-102,392,302",-0.944851,-0.977350
athe,83.556223,48,0,"424,797",278,-47,"-424,518",-0.992301,-0.999345
...,...,...,...,...,...,...,...,...,...
vall,9042.701236,"6,149",0,"28,124,581",0,"-6,149","-28,124,581",-0.999990,-1.000000
walc,3898.844618,"10,062","2,328","3,099,922","379,502","-7,733","-2,720,420",-0.768588,-0.877577
wind,1296.603117,"2,226","3,177","2,916,536","8,217,436",951,"5,300,900",0.427290,1.817533


,COUNTY_ID,ACRES,units_10,units_18,sf_10,sf_18,unit_diff,sqft_diff,unit_diff_pct,sqft_diff_pct,ctyName
0,1.0,230468.562801,"73,908","126,658","323,375,208","321,412,855","52,749","-1,962,352",0.713722,-0.006068,Alameda
1,13.0,239812.143402,"159,961","54,193","394,103,264","190,014,128","-105,767","-204,089,135",-0.661207,-0.517857,Contra Costa
2,41.0,221327.899567,"22,343","184,578","20,638,668","228,373,302","162,234","207,734,634",7.260775,10.065312,Marin
3,55.0,95917.110035,"22,743","8,073","98,109,285","255,710,822","-14,670","157,601,536",-0.645028,1.606388,Napa
4,75.0,2012.375580,"5,826","2,805","43,318,064","3,104,803","-3,020","-40,213,261",-0.518450,-0.928325,San Francisco
5,81.0,200711.272259,"33,430","38,837","148,997,326","158,989,343","5,406","9,992,017",0.161711,0.067062,San Mateo
6,85.0,489742.499923,"219,027","49,338","1,823,221,928","235,923,973,760","-169,689","234,100,751,831",-0.774740,128.399482,Santa Clara
7,95.0,288686.289781,"43,417","15,449","268,216,698","129,258,508","-27,967","-138,958,190",-0.644153,-0.518082,Solano
8,97.0,481874.669405,"44,217","16,560","377,832,997","58,699,066","-27,657","-319,133,930",-0.625482,-0.844643,Sonoma


In [14]:
# low ILR parcel statistics (threadhold 0.2)
plu_main.ILR = pd.to_numeric(plu_main.ILR, errors='coerce')
p_low_ILR = plu_main.loc[plu_main.ILR < 0.2]

low_ILR_boc_j = boc_j(p_low_ILR)
low_ILR_boc_j.to_csv('low_ILR_boc_jurisdiction.csv')

low_ILR_boc_c = boc_c(p_low_ILR)
low_ILR_boc_c.to_csv('low_ILR_boc_county.csv')

,ACRES,units_10,units_18,sf_10,sf_18,unit_diff,sqft_diff,unit_diff_pct,sqft_diff_pct
juris_id_y,,,,,,,,,
alam,229.266511,"1,019",269,"9,446,845",0,-749,"-9,446,845",-0.735265,-1.000000
alba,28.675618,529,0,"1,010,220","34,411",-529,"-975,809",-1.000000,-0.965937
anti,7610.578627,"28,829","1,122","110,091,277","1,939,731","-27,706","-108,151,545",-0.961059,-0.982381
athe,436.792943,411,31,"3,444,549","27,204",-379,"-3,417,345",-0.923297,-0.992102
belm,358.951603,825,"1,096","6,959,931","1,546,063",271,"-5,413,868",0.328786,-0.777862
...,...,...,...,...,...,...,...,...,...
vaca,4759.994793,"6,547",983,"23,774,539","28,808,207","-5,564","5,033,668",-0.849750,0.211725
vall,2188.668326,"3,364",0,"18,145,355",0,"-3,364","-18,145,355",-1.000000,-1.000000
walc,4058.807474,"6,300","1,746","5,611,717","906,660","-4,554","-4,705,057",-0.722783,-0.838434


,COUNTY_ID,ACRES,units_10,units_18,sf_10,sf_18,unit_diff,sqft_diff,unit_diff_pct,sqft_diff_pct,ctyName
0,1.0,175765.465815,"116,447","91,803","456,058,249","148,357,909","-24,644","-307,700,339",-0.211637,-0.674695,Alameda
1,13.0,280997.464041,"128,363","53,337","360,363,278","226,404,831","-75,026","-133,958,446",-0.584485,-0.371732,Contra Costa
2,41.0,74589.311165,"36,409","136,018","24,237,326","360,252,769","99,609","336,015,443",2.735828,13.863553,Marin
3,75.0,878.698191,"15,041","4,305","74,799,121","3,522,425","-10,736","-71,276,695",-0.713760,-0.952908,San Francisco
4,81.0,118639.711196,"31,575","30,381","198,002,023","94,471,560","-1,193","-103,530,462",-0.037807,-0.522876,San Mateo
5,85.0,371116.819941,"139,924","23,099","1,744,657,885","194,419,140,729","-116,824","192,674,482,844",-0.834913,110.436828,Santa Clara
6,95.0,304074.167364,"55,474","8,749","339,664,383","98,736,240","-46,725","-240,928,143",-0.842279,-0.709312,Solano
7,97.0,498976.883230,"46,085","12,561","310,599,829","26,907,600","-33,524","-283,692,229",-0.727438,-0.913369,Sonoma


In [15]:
# Old building parcel statistics (1930-1980)
plu_main.year_built = pd.to_numeric(plu_main.year_built, errors='coerce')
p_old = plu_main.loc[(plu_main.year_built < 1980) & (plu_main.year_built >= 1930)]

old_boc_j = boc_j(p_old)
old_boc_j.to_csv('old_boc_jurisdiction.csv')

old_boc_c = boc_c(p_old)
old_boc_c.to_csv('old_boc_county.csv')

,ACRES,units_10,units_18,sf_10,sf_18,unit_diff,sqft_diff,unit_diff_pct,sqft_diff_pct
juris_id_y,,,,,,,,,
alam,3464.036126,"13,079","4,719","16,464,241",0,"-8,359","-16,464,241",-0.639140,-1.000000
alba,2009.003610,"6,266",0,"11,316,266","339,100","-6,266","-10,977,166",-1.000000,-0.970034
amer,363.616684,"1,125",141,"943,056","2,315,824",-983,"1,372,768",-0.874171,1.455658
anti,2692.749556,"20,044","2,622","24,863,565","1,143,905","-17,421","-23,719,659",-0.869144,-0.953993
athe,2022.486361,"1,961",157,"15,309,359","157,032","-1,804","-15,152,327",-0.919871,-0.989743
...,...,...,...,...,...,...,...,...,...
vall,5043.332009,"27,504",0,"38,460,202",0,"-27,504","-38,460,202",-1.000000,-1.000000
walc,5845.449394,"29,404","9,290","34,165,032","4,792,567","-20,113","-29,372,464",-0.684048,-0.859723
wind,954.799128,"3,377","6,632","2,348,254","11,643,419","3,255","9,295,165",0.963840,3.958330


,COUNTY_ID,ACRES,units_10,units_18,sf_10,sf_18,unit_diff,sqft_diff,unit_diff_pct,sqft_diff_pct,ctyName
0,1.0,114291.727364,"567,191","297,497","976,046,776","600,674,905","-269,693","-375,371,871",-0.475490,-0.384584,Alameda
1,13.0,111017.554884,"346,267","125,013","358,832,465","320,125,941","-221,253","-38,706,523",-0.638969,-0.107868,Contra Costa
2,41.0,127469.517135,"140,282","257,410","132,606,622","696,477,279","117,128","563,870,656",0.834945,4.252206,Marin
3,55.0,311599.693557,"317,660","15,610","270,586,520","188,726,635","-302,050","-81,859,884",-0.950858,-0.302528,Napa
4,75.0,9176.979954,"185,929","47,376","685,096,316","20,351,300","-138,553","-664,745,016",-0.745190,-0.970294,San Francisco
5,81.0,112244.589138,"281,996","115,261","533,622,229","242,709,991","-166,735","-290,912,238",-0.591267,-0.545165,San Mateo
6,85.0,218281.371334,"506,501","263,872","3,173,710,657","165,743,696,159","-242,629","162,569,985,501",-0.479030,51.223947,Santa Clara
7,95.0,167993.985438,"87,001","37,596","204,775,655","111,316,613","-49,404","-93,459,042",-0.567859,-0.456397,Solano
8,97.0,312131.304041,"140,886","84,466","734,076,757","68,020,543","-56,420","-666,056,214",-0.400465,-0.907339,Sonoma
